# TAP

This notebook is for inspecting timelapse microscopy data, with associated sinhgle-cell labels and tracks, showing the infection of human macrophages with Mycobacterium Tuberculosis (Mtb), acquired on an Opera Phenix confocal microscope. 

In [1]:
import napari
from macrohet import dataio, tile, visualise
import os, glob
from skimage import io
import tarrow
import torch
import numpy as np
import tifffile
import dask.array as da

# Load image

In [2]:
images = da.from_zarr('/mnt/DATA/macrohet/macrohet_images/ND0000/zarr/2.10_gfp_rfp.zarr')
images

dask.array<from-zarr, shape=(376, 2, 3, 6048, 6048), dtype=uint16, chunksize=(1, 2, 3, 6048, 6048), chunktype=numpy.ndarray>

# Apply model

In [4]:
model = tarrow.models.TimeArrowNet.from_folder(model_folder='/home/dayn/analysis/models/tarrow/ND0000/macro_backbone_unet/')

INFO:root:Loading model from /home/dayn/analysis/models/tarrow/ND0000/macro_backbone_unet


In [6]:
input_image = da.sum(images, axis = 2)[0,0].compute().astype(np.float32)

input_image = input_image.reshape((1, 1, 1, input_image.shape[0], input_image.shape[1]))

input_tensor = torch.Tensor(input_image)

In [7]:
input_tensor.shape

torch.Size([1, 1, 1, 6048, 6048])

In [16]:
%%time
rep = model.embedding(input_tensor)
print(f"Dense representations for image volume with shape {tuple(rep.shape)}")

Dense representations for image volume with shape (1, 1, 32, 6048, 6048)
CPU times: user 2h 9min 20s, sys: 2min 15s, total: 2h 11min 36s
Wall time: 7min 44s


In [17]:
np_rep = rep[0,0,...].detach().numpy()

In [18]:
np_rep.shape

(32, 6048, 6048)

In [43]:
viewer.add_image(np_rep, channel_axis=0, blending='additive')

[<Image layer 'Image' at 0x7fdf7256f850>,
 <Image layer 'Image [1]' at 0x7fdf72597850>,
 <Image layer 'Image [2]' at 0x7fdf722d20d0>,
 <Image layer 'Image [3]' at 0x7fdf7c580430>,
 <Image layer 'Image [4]' at 0x7fdf684d25b0>,
 <Image layer 'Image [5]' at 0x7fdf66239880>,
 <Image layer 'Image [6]' at 0x7fdf79c0fb50>,
 <Image layer 'Image [7]' at 0x7fdf74e9e610>,
 <Image layer 'Image [8]' at 0x7fdf7c8e5880>,
 <Image layer 'Image [9]' at 0x7fdf88752df0>,
 <Image layer 'Image [10]' at 0x7fdf8877fe20>,
 <Image layer 'Image [11]' at 0x7fdf88730d30>,
 <Image layer 'Image [12]' at 0x7fdf886dcd60>,
 <Image layer 'Image [13]' at 0x7fdf8870cc70>,
 <Image layer 'Image [14]' at 0x7fdf886b8ca0>,
 <Image layer 'Image [15]' at 0x7fdf88669bb0>,
 <Image layer 'Image [16]' at 0x7fdf88614be0>,
 <Image layer 'Image [17]' at 0x7fdf88646af0>,
 <Image layer 'Image [18]' at 0x7fdf885f0b20>,
 <Image layer 'Image [19]' at 0x7fdf63554a30>,
 <Image layer 'Image [20]' at 0x7fdf63500a60>,
 <Image layer 'Image [21]' 

In [19]:
np_rep_sum = np.sum(np_rep, axis = 0)

In [20]:
viewer = napari.Viewer()
viewer.add_image(input_image, blending = 'additive')
viewer.add_image(np_rep_sum, blending = 'additive')

<Image layer 'np_rep_sum' at 0x7fb0eb5d6d00>

In [21]:
viewer.add_image(np_rep)

<Image layer 'np_rep' at 0x7fb0eaca4be0>

# Run over all frames?

In [22]:
from tqdm.auto import tqdm

In [24]:
images = da.sum(images, axis = 2)[:,0]

In [25]:
images

dask.array<getitem, shape=(376, 6048, 6048), dtype=uint64, chunksize=(1, 6048, 6048), chunktype=numpy.ndarray>

In [ ]:
import numpy as np
import torch
from tqdm.auto import tqdm

# Define the step size (e.g., every tenth frame)
step_size = 1
num_frames = len(images)
rep_sums = []
channel = 0

for t in tqdm(range(0, num_frames, step_size), total=num_frames // step_size):
    input_image = images[t].compute().astype(np.int16)
    input_tensor = torch.Tensor(input_image[None, None, None])

    with torch.no_grad():
        rep = model.embedding(input_tensor)
    
    np_rep = rep[0, 0, ...].cpu().numpy()
    np_rep_sum = np.sum(np_rep, axis=0)
    rep_sums.append(np_rep_sum)

rep_sums = np.stack(rep_sums, axis=0)


  0%|          | 0/376 [00:00<?, ?it/s]

In [16]:
viewer = napari.Viewer(title = 'tarrow over every frame')
viewer.add_image(selected_frames_volume, blending = 'additive')
viewer.add_image(rep_sums, blending = 'additive')

<Image layer 'rep_sums' at 0x7f5669c91670>

In [17]:
np.save('/home/dayn/analysis/models/tarrow/reduced_batchsize/macro_backbone_unet/visuals/all_frames.npy', selected_frames)
np.save('/home/dayn/analysis/models/tarrow/reduced_batchsize/macro_backbone_unet/visuals/all_frames_rep_sums.npy', rep_sums)

In [2]:
rep_sums = np.load('/home/dayn/analysis/models/tarrow/reduced_batchsize/macro_backbone_unet/visuals/all_frames_rep_sums.npy')

In [3]:
rep_sums.shape

(75, 6048, 6048)

In [4]:
viewer = napari.Viewer()
viewer.add_image(rep_sums)

<Image layer 'rep_sums' at 0x7f07b269b5b0>

# MDCK example

In [7]:
mdck = images = io.imread('/home/dayn/analysis/tarrow/scripts/data/mdck.tif')

In [6]:
model = tarrow.models.TimeArrowNet.from_folder(model_folder='/home/dayn/analysis/tarrow/scripts/data/runs/10-20-16-56-38_None_backbone_unet/')

INFO:root:Loading model from /home/dayn/analysis/tarrow/scripts/data/runs/10-20-16-56-38_None_backbone_unet


In [ ]:
import numpy as np
import torch
from tqdm.auto import tqdm

# Define the step size (e.g., every tenth frame)
step_size = 1
num_frames = len(images)
rep_sums = []
channel = 0

for t in tqdm(range(0, num_frames, step_size), total=num_frames // step_size):
    input_image = images[t]#.compute().astype(np.int16)
    input_tensor = torch.Tensor(input_image[None, None, None])

    with torch.no_grad():
        rep = model.embedding(input_tensor)
    
    np_rep = rep[0, 0, ...].cpu().numpy()
    np_rep_sum = np.sum(np_rep, axis=0)
    rep_sums.append(np_rep_sum)

rep_sums = np.stack(rep_sums, axis=0)


  0%|          | 0/1194 [00:00<?, ?it/s]

In [ ]:
np.save('/home/dayn/analysis/tarrow/scripts/data/mdck_reps.npy')

In [ ]:
print()